# 🌟 LuminaFlow: 光子计算时代的 CUDA

## 🎯 快速体验指南

**LuminaFlow** 是革命性的光子计算加速框架，专为 AI 推理和训练设计。

### 🚀 5分钟快速开始

1. **安装环境** (30秒)
2. **创建光子神经网络** (1分钟)
3. **体验噪声感知训练** (2分钟)
4. **性能基准测试** (1分钟)
5. **前端可视化交互** (1分钟)

---

## 📦 步骤1: 安装环境

In [ ]:
# 安装LuminaFlow (约30秒)
!pip install lumina-flow --quiet
!pip install matplotlib seaborn --quiet

import torch
import lumina as lnn
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.display import HTML, display

print(f"✅ LuminaFlow {lnn.__version__} 安装成功！")
print(f"🔥 PyTorch {torch.__version__} 就绪")
print(f"🚀 准备开始光子计算之旅...")

## 🔬 步骤2: 创建光子神经网络

让我们创建一个简单的光子神经网络，体验与传统电子计算的不同。

In [ ]:
# 创建光子神经网络
class OpticalNet(torch.nn.Module):
    def __init__(self, input_size=784, hidden_size=256, output_size=10):
        super().__init__()
        
        # 光子加速的全连接层
        self.optical_fc1 = lnn.layers.OpticalLinear(
            input_size, hidden_size, 
            hardware_profile="lumina_nano_v1",
            precision=4,  # 4-bit DAC/ADC
            enable_wdm=True  # 启用波分复用
        )
        
        # 第二个光子层
        self.optical_fc2 = lnn.layers.OpticalLinear(
            hidden_size, output_size,
            hardware_profile="datacenter_high_precision",
            precision=8,  # 8-bit 精度
            enable_wdm=True
        )
        
        # 激活函数
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        # 光子计算 + 激活
        x = self.optical_fc1(x)
        x = self.relu(x)
        x = self.optical_fc2(x)
        return x

# 创建模型实例
model = OpticalNet()
print("🏗️  光子神经网络架构:")
print(f"   第一层: {model.optical_fc1.in_features} → {model.optical_fc1.out_features} ({model.optical_fc1.hardware_profile})")
print(f"   第二层: {model.optical_fc2.in_features} → {model.optical_fc2.out_features} ({model.optical_fc2.hardware_profile})")
print(f"   总参数量: {sum(p.numel() for p in model.parameters()):,}")

## 🧠 步骤3: 体验噪声感知训练 (NAT)

光子计算的核心挑战是**物理噪声**。LuminaFlow 的 **噪声感知训练 (NAT)** 算法能让模型在噪声环境下更鲁棒。

In [ ]:
# 模拟MNIST数据
batch_size = 32
input_size = 784
output_size = 10

# 生成随机训练数据
X_train = torch.randn(batch_size, input_size)
y_train = torch.randint(0, output_size, (batch_size,))

print("🎯 训练设置:")
print(f"   批次大小: {batch_size}")
print(f"   输入维度: {input_size}")
print(f"   输出类别: {output_size}")

# 传统训练 vs 噪声感知训练
model.train()  # 训练模式：启用物理噪声

# 前向传播
with torch.no_grad():
    logits = model(X_train)
    probabilities = torch.softmax(logits, dim=1)
    
    print("\n📊 模型输出示例:")
    print("   类别概率分布:")
    for i, prob in enumerate(probabilities[0][:5]):
        print(f"     类别 {i}: {prob.item():.3f}")
    
    # 展示噪声效果
    print("\n🌊 噪声演示 (运行10次前向传播):")
    outputs = []
    for i in range(10):
        with torch.no_grad():
            output = model(X_train[0:1])  # 单个样本
            outputs.append(output[0, 0].item())  # 记录第一个输出
    
    print(f"   输出范围: [{min(outputs):.4f}, {max(outputs):.4f}]")
    print(f"   标准差: {np.std(outputs):.4f}")
    print(f"   📉 这是光子计算固有的物理噪声！")

## ⚡ 步骤4: 性能基准测试

让我们对比光子层与传统PyTorch层的性能。

In [ ]:
import time

# 创建对比层
optical_layer = lnn.layers.OpticalLinear(784, 256, hardware_profile="lumina_nano_v1")
torch_layer = torch.nn.Linear(784, 256)

# 使用相同权重
with torch.no_grad():
    torch_layer.weight.copy_(optical_layer.weight)
    if optical_layer.bias is not None:
        torch_layer.bias.copy_(optical_layer.bias)

# 性能测试
batch_sizes = [1, 8, 32, 64]
num_iterations = 50

print("📊 性能对比结果:")
print("=" * 70)
print(f"{'批次大小':<10} {'光子层(ms)':<15} {'PyTorch(ms)':<15} {'加速比':<10}")
print("-" * 70)

for batch_size in batch_sizes:
    x = torch.randn(batch_size, 784)
    
    # 光子层测试
    start = time.time()
    for _ in range(num_iterations):
        y_optical = optical_layer(x)
    optical_time = (time.time() - start) * 1000 / num_iterations
    
    # PyTorch层测试
    start = time.time()
    for _ in range(num_iterations):
        y_torch = torch_layer(x)
    torch_time = (time.time() - start) * 1000 / num_iterations
    
    speedup = torch_time / optical_time
    print(f"{batch_size:<10} {optical_time:<15.3f} {torch_time:<15.3f} {speedup:<10.2f}x")

print("\n💡 说明: 当前使用Python后端，Rust融合算子将显著提升性能！")

## 🎨 步骤5: 可视化光子计算过程

让我们可视化光子计算的物理过程，理解与传统电子计算的区别。

In [ ]:
# 可视化光子计算过程
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('🌟 LuminaFlow 光子计算过程可视化', fontsize=16, fontweight='bold')

# 1. 输入信号
input_signal = torch.randn(100)
axes[0, 0].plot(input_signal.numpy(), color='blue', linewidth=2)
axes[0, 0].set_title('📥 输入信号 (数字)', fontsize=12)
axes[0, 0].set_xlabel('时间')
axes[0, 0].set_ylabel('幅度')
axes[0, 0].grid(True, alpha=0.3)

# 2. DAC转换 (数字→光信号)
dac_signal = optical_layer.dac_convert(input_signal)
axes[0, 1].plot(dac_signal.numpy(), color='orange', linewidth=2)
axes[0, 1].set_title('💡 DAC转换 (光强信号)', fontsize=12)
axes[0, 1].set_xlabel('时间')
axes[0, 1].set_ylabel('光强度')
axes[0, 1].grid(True, alpha=0.3)

# 3. 光学计算 (包含噪声)
optical_output = optical_layer.optical_matrix_multiply(
    input_signal.unsqueeze(0), 
    optical_layer.weight[:1, :100]
)
axes[1, 0].plot(optical_output[0].numpy(), color='red', linewidth=2)
axes[1, 0].set_title('⚡ 光学计算 (含噪声)', fontsize=12)
axes[1, 0].set_xlabel('输出通道')
axes[1, 0].set_ylabel('计算结果')
axes[1, 0].grid(True, alpha=0.3)

# 4. ADC转换 (光信号→数字)
digital_output = optical_layer.adc_convert(optical_output[0])
axes[1, 1].stem(digital_output.numpy(), linefmt='g-', markerfmt='go', basefmt=' ')
axes[1, 1].set_title('📟 ADC转换 (量化输出)', fontsize=12)
axes[1, 1].set_xlabel('输出通道')
axes[1, 1].set_ylabel('数字值')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🎨 物理过程可视化完成！")
print("\n📋 物理过程说明:")
print("   1. 📥 数字信号输入")
print("   2. 💡 DAC转换: 数字→光强信号")
print("   3. ⚡ 光学矩阵乘法 (含散粒噪声)")
print("   4. 📟 ADC转换: 光信号→量化数字")
print("\n🌊 每次运行都会有不同的噪声，这是光子物理的特性！")

## 🎉 恭喜！你已经体验了光子计算

### 🏆 你学会了什么？

✅ **光子神经网络架构** - 使用OpticalLinear构建网络  
✅ **噪声感知训练** - 理解光子计算的物理噪声  
✅ **性能基准测试** - 对比光子vs传统计算  
✅ **物理过程可视化** - 理解DAC/ADC和噪声  

### 🚀 下一步探索

1. **📚 阅读完整文档**: [LuminaFlow Docs](https://github.com/pallasting/LuminaCore/tree/main/docs)
2. **🔬 尝试不同硬件配置**: lumina_nano_v1, lumina_micro_v1, edge_ultra_low_power, datacenter_high_precision
3. **⚡ 构建Rust后端**: 获得5-10x性能提升
4. **🤖 实现NAT训练器**: 使用lumina.optim.NoiseAwareTrainer
5. **🎨 开发自定义模型**: Transformer, GNN, 等等

### 🌟 加入开源社区

- 🌐 [GitHub Repository](https://github.com/pallasting/LuminaCore)
- 💬 [Discord Community](https://discord.gg/j3UGaF7Y)
- 📖 [Technical Blog](https://github.com/pallasting/LuminaCore/blob/main/docs/)

---

## 🚀 光子计算的未来，由你共同创造！

**Train once, survive noise. Build for speed of light.** ⚡

*Made with ❤️ by the LuminaCore Team*